In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, derived = load_pickles()

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Constants
C = 4.67e-13 # TODO: where is this constant from? Sara's email (and PDF) 29/08
lambda0  = [6301.51*1e-8, 6302.50*1e-8]   # Angstroms to cm
gbar     = [1.669, 2.487] # Lozitsky

In [ ]:
def noise_level(a):
    a = np.asanyarray(a)
    sd = a.std(ddof=1)
    return sd

# Example spectrum noise level
spectrum = copy.copy(V.data_n[300,300,:])
sd = noise_level(spectrum[90:])
print(f'Noise level = {sd}')
plt.plot(V.wave_array, spectrum, linestyle='-')
plt.hlines(sd, V.wave_array[0], V.wave_array[-1], color='red')

In [ ]:
dlB = np.zeros(I.data.shape)[:,:,:2].astype(float)
margin = 3.5

for i in range(400,403):
    for j in range(1,2):
        print(f'For pixel {i},{j}:')
        peaks_p = [0, 0]
        peaks_n = [0, 0]

        spectrum = copy.copy(V.data_n[i,j,:])

        sd = noise_level(spectrum[90:]) # Calculate noise level for spectrum region outside the spectral lines
        print(f'Noise level = {sd}')

        # First line
        peaks_p[0] = int(np.argmax(spectrum[:60])) if spectrum[:60].max() > margin*sd else None
        peaks_n[0] = int(np.argmin(spectrum[:60])) if spectrum[:60].min() < -margin*sd else None

        # # Second line
        peaks_p[1] = 60 + int(np.argmax(spectrum[60:])) if spectrum[60:].max() > margin*sd else None
        peaks_n[1] = 60 + int(np.argmin(spectrum[60:])) if spectrum[60:].min() < -margin*sd else None

        print(f'Peak positions: {peaks_p}, {peaks_n}')


        plt.figure()
        if all(x is not None for x in peaks_p) and all(x is not None for x in peaks_n):
            plt.vlines(V.wave_array[peaks_n], ymin=np.min(spectrum), ymax=np.max(spectrum), color='g', linestyle='--')
            plt.vlines(V.wave_array[peaks_p], ymin=np.min(spectrum), ymax=np.max(spectrum), color='purple', linestyle='--')

            print(f'Peaks positive: {V.wave_array[peaks_p]}')
            print(f'Peaks negative: {V.wave_array[peaks_n]}')
            print(np.array(V.wave_array)[peaks_p] - np.array(V.wave_array)[peaks_n])
            a = (np.array(V.wave_array)[peaks_p] - np.array(V.wave_array)[peaks_n])
            dlB[i, j, :] = np.array(V.wave_array)[peaks_p] - np.array(V.wave_array)[peaks_n]
            print(f'Distances: {dlB[i, j, 0]:.6f}, {dlB[i, j, 1]:.6f} Angstrom')

        plt.hlines(margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
        plt.hlines(-margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
        plt.plot(V.wave_array, spectrum, linestyle='-', color='black')



        # else:
            # print(f'Found {np.size(peaks_p)} positive peaks and {np.size(peaks_n)} negative peaks. Not calculating distances.')

In [ ]:
(I.size_y)

In [ ]:
plot_data(dlB[:,:,1])

In [ ]:
I.data.shape